# PAPERS DECONSTRUCTED
## Title: XGBoost: A Scalable Tree Boosting System
### Authors: Tianqi Chen, Carlos Guestrin
Link: https://arxiv.org/abs/1603.02754

# Executive Summary

The core idea is to take the original <a href="https://projecteuclid.org/journals/annals-of-statistics/volume-29/issue-5/Greedy-function-approximation-A-gradient-boostingmachine/10.1214/aos/1013203451.full">work</a> of Friedman for boosting algorithms with the following improvements:

(1) Add regularization into the framework explicitly

(2) Add a bunch of computer science optimiations:

    - Make finding split points more efficient
    - Make sparsity a first-class citizen
    - Improve how sorting a feature is done
    - Caching access of gradient statistics

# Core Algorithm

The idea is to use a Taylor series approximation of the loss function and just work off of that. For a single valued function, a Taylor series looks like:

$f(x) = f(x_0) + f'(x_0)(x - x_0) + \frac{1}{2}f''(x_0)(x - x_0)^2$

In our case, the loss function is a function of two values: $l(y, \hat{y})$. In boosting, $\hat{y}$ is a sum of all previous boosts:

$\hat{y}_m = f_m(x) + F_{m-1}(x)$

Where $F_{m-1}(x) = f_{m-1}(x) + f_{m-2}(x) + \dots$

Let's look at our loss:

$J(y, \hat{y}) = \sum_i l(y, \hat{y})$

But the authors also make the point that they want to add some reguarlization to each boost. So we get:

$J(y, f_m(x) + F_{m-1}(x)) = \sum_i^N l(y, \hat{y}) + \sum_k^{m} \Omega(f_k(x))$

So it's assumed each example is independent. Now, we want to write a Taylor series approximation of the loss function. So let's focus on:

$l(y, \hat{y}) = l(y, f_m(x) + F_{m-1}(x))$

If we assume a stage-wise approach (where all previous terms cannot be modified). Then $f_m(x)$ is like our $x$ variable we want to expand over just like in the one-variable Taylor series. So we get: 

$l(y, f_m(x) + F_{m-1}(x)) \approx l(y, F_{m-1}(x)) + \frac{\partial l}{\partial f}_{f = F_{m-1}(x)} f_m(x) + \frac{1}{2} \frac{\partial^2 l}{\partial f^2} (f_m(x))^2$

Plugging this back into the main equation:

$J(y, \hat{y}) = \sum_i^N \big( l(y_i, F_{m-1}(x_i)) + \frac{\partial l}{\partial f}_{f = F_{m-1}(x_i)} f_m(x_i) + \frac{1}{2} \frac{\partial^2 l}{\partial f^2} (f_m(x))^2 \big) + \sum_k^{m} \Omega(f_k(x))$

We want to find a $f_m(x_i)$ such that it minimizes this cost function. This means off the bat, we can ignore $l(y_i, F_{m-1}(x_i))$

So now we get:

$\tilde{J}(y, \hat{y}) = \sum_i^N \big( \frac{\partial l}{\partial f}_{f = F_{m-1}(x_i)} f_m(x_i) + \frac{1}{2} \frac{\partial^2 l}{\partial f^2} (f_m(x))^2 \big) + \sum_k^{m} \Omega(f_k(x))$

$f_m(x)$ takes an example and directs it to a leaf (assuming CART is our booster). This mapping is unique. So we can think of this as a vector of leafs. This vector will have zero components everwhere except at the index $x$ maps to. It's length is the number of leafs.

So we have $f_m(x_i) = \vec{w_j}(x_i)$

The trick here is that we always take the non-zero component.

So our entire example space can be partitioned into the leaves that will map to. So we can add a new summation term that sums the examples by leaf. Let's say we have T leafs:


$\tilde{J} = \sum_j^T \sum_{i \in R_j} \big( \frac{\partial l}{\partial f}_{f = F_{m-1}(x_i)} \vec{w_j} + \frac{1}{2} \frac{\partial^2 l}{\partial f^2} (\vec{w_j})^2 \big) + \sum_j^{T} \Omega(f_{j}(x))$

The last term can be simplifed since all previous trees are frozen (i.e., they are constants). This means $\Omega(f_{jk}(x)) = \Omega(f_m(x))$

Now, how exactly do we want to quantify the regularaization? The authors propose that the number of leafs and their values are the things we should control from unregulated growth. 

So $\Omega(f_m(x)) = \gamma T + \frac{1}{2} \lambda \sum_j \vec{w}^2_j$

Plugging this all back in:


$\tilde{J} = \sum_j^T \sum_{i \in R_j} \big( \frac{\partial l}{\partial f}_{f = F_{m-1}(x_i)} \vec{w_j} + \frac{1}{2} \frac{\partial^2 l}{\partial f^2} \vec{w_j}^2 \big) +  \gamma T + \frac{1}{2} \lambda \sum_j \vec{w}^2_j$

We can rearange sum:

$\tilde{J} = \sum_j^T \big(  \vec{w_j} \sum_{i \in R_j} \frac{\partial l}{\partial f}_{f = F_{m-1}(x_i)} + \frac{1}{2} \vec{w_j}^2 \sum_{i \in R_j} \frac{\partial^2 l}{\partial f^2} +  \frac{1}{2} \lambda \vec{w}^2_j \big) +  \gamma T$

We can make some defitions to clean things up:

$G_i = \sum_{i \in R_j} \frac{\partial l}{\partial f}_{f = F_{m-1}(x_i)}$

$H_i = \sum_{i \in R_j} \frac{\partial^2 l}{\partial f^2}_{f = F_{m-1}(x_i)}$


$\tilde{J} = \sum_j^T \big(  \vec{w_j} G_i + \frac{1}{2} \vec{w_j}^2 H_i +  \frac{1}{2} \lambda \vec{w}^2_j \big) +  \gamma T$

Things are starting to look better!

$\tilde{J} = \sum_j^T \big(  \vec{w_j} G_i + \frac{1}{2} \vec{w_j}^2 (H_i + \lambda) \big) +  \gamma T$

To find the lowest, for a fixed set of leafs, we need to minimize:

$\sum_j^T \big(  \vec{w_j} G_i + \frac{1}{2} \vec{w_j}^2 (H_i + \lambda) \big)$

Each leaf will not share examples with other leafs. So for each leaf, we want to make 

$\vec{w_j} G_i + \frac{1}{2} \vec{w_j}^2 (H_i + \lambda)$ as small as possible. Taking its derivative with respect to $\vec{w_j}$ and setting to 0:

$G_i + \vec{w_j} (H_i + \lambda) = 0$

Now, we can solve for $\vec{w_j}$

$\vec{w_j} = -\frac{G_i}{H_i + \lambda}$